# 第八章：类与对象

本章主要关注点的是和类定义有关的常见编程模型。包括让对象支持常见的Python特性、特殊方法的使用、类封装技术、继承、内存管理以及有用的设计模式。

## 8.1 改变对象的字符串显示

* 问题

改变对象实例的打印或显示输出，让它们更具可读性。

* 解决方案

要改变一个实例的字符串表示，可重新定义它的 `__str__()` 和 `__repr__()` 方法。

In [2]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

`__repr__()` 返回一个实例的代码表示形式，通常用来构造这个实例。内置的 `repr()` 函数返回这个字符串，跟我们使用交互式解释器显示的值是一样的。

`__str__()` 将实例转换为一个字符串，使用 `str()` 或 `print()` 函数会输出这个字符串。

In [3]:
p = Pair(3, 4)
p  # __repr__() output

Pair(3, 4)

In [5]:
print(p)  # __str__() output

(3, 4)


上述代码还演示了再格式化的时候怎样使用不同的字符串表现形式。

特别来讲，`!r` 格式化代码指明输出使用 `__repr__()` 来代替默认的 `__str()__`。

可以使用Pair类测试

In [6]:
p = Pair(3, 4)
print('p is {0!r}'.format(p))

p is Pair(3, 4)


In [7]:
print('p is {0}'.format(p))

p is (3, 4)


* 讨论

自定义 `__repr__()` 和 `__str()__` 可以简化调试和实例输出。我们只用通过打印实例，程序员就会看到实例更加详细的信息

`__repr__()` 生成的文本字符串标准做法是需要让 `eval(repr(x)) == x` 为真。


如果不行或者不希望有这种行为，那么通常就让它产生一段有帮助意义的文本，并且以 `<` 和 `>` 括起来

In [ ]:
f = open('file.dat')

如果 `__str__()` 没有被定义，那么就会使用 `__repr__()` 来代替输出。

上面的 `format()` 格式化方法 `{0.x}` 对应的是第1个参数的x属性。因此，在下面的函数中，0实际上指的就是 self 本身：
```python
def __repr__(self):
    return 'Pair({0.x!r}, {0.y!r})'.format(self)
```

作为这种实现的一个替代，你也可以使用 `%` 操作符，就像下面这样：
```python
def __repr__(self):
    return 'Pair(%r, %r)' % (self.x, self.y)
```

## 8.2 自定义字符串的格式化

* 问题

通过 `format()` 函数和字符串方法使得一个对象能支持自定义的格式化。

* 解决方案

为了自定义字符串的格式化，我们需要在类上面定义 `__format__()` 方法。

In [8]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

In [14]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        print(self)
        return fmt.format(d=self)

In [15]:
# 现在 `Date` 类的实例可以支持格式化操作了
d = Date(2012, 12, 21)
format(d)

'2012-12-21'

In [16]:
format(d, 'mdy')

'12/21/2012'

In [12]:
'The date is {:ymd}'.format(d)

'The date is 2012-12-21'

In [13]:
'The date is {:mdy}'.format(d)

'The date is 12/21/2012'

* 讨论

`__format__()` 方法给Python的字符串格式化功能提供了一个钩子。

这里需要着重强调的是格式化代码的解析工作完全由类自己决定。因此，格式化代码可以是任何值。

例如，参考下面来自 datetime 模块中的代码：

In [17]:
from datetime import date

d = date(2012, 12, 21)
format(d)

'2012-12-21'

In [18]:
format(d, '%A, %B%d, %Y')

'Friday, December21, 2012'

In [19]:
'The end is {:%d %b %Y}. Goodbye'.format(d)

'The end is 21 Dec 2012. Goodbye'

对于内置类型的格式化有一些标准的约定。 可以参考 [string模块文档](https://docs.python.org/3/library/string.html) 说明。

## 8.3 让对象支持上下文管理协议

* 问题

让对象支持上下文管理协议(with语句)。

* 解决方案

为了让一个对象兼容 `with` 语句，你需要实现 `__enter__()` 和 `__exit__()` 方法。

In [20]:
# 假设创建一个能够创建网络连接的类：
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

这个类的核心功能就是表示一条网络连接，但是初始化的时候并不会做任何事情（比如它并没有建立一个连接）。连接的建立和关闭是使用 `with` 语句自动完成的。

In [ ]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed